# Webscraping da CVM

Esse robô será feito com as seguintes etapas:

1. Função init


2. Função webscraping

    a. Verificação do diretório de destino e limpeza, caso seja necessário
    
    b. Acionar o nosso webdriver
    
    c. Determinar o intervalo de tempo que queremos baixar os arquivos
    
    d. Solicitar que o webdriver baixe os arquivos
    
    e. Vamos retirar os arquivos .csv de dentro das pastas zipadas e vamos apagar as pastas zipadas
    
    
3. Função importação

    a. Determinar quais arquivos serão abertos
    
    b. Concatenar tudo num data frame só
    
    c. Fazer adaptações necessárias e retornar o data frame final
    
    


# 1. Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import zipfile
import pandas as pd
import os

# 2. Classe

In [7]:
class CVM_Extractor:

    def __init__(self, ini, fim):
        self.dir_download = os.getcwd() + '\\files' # Diretório onde está o script
        self.url = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
        self.ini = ini
        self.fim = fim

    def scraping_df(self):
        if os.path.isdir(self.dir_download): #Verificamos se o diretório existe
            print("Diretório Existe")
            # Caso o diretório exista, vamos limpá-lo se houver arquivos
            if(len(os.listdir(self.dir_download))>0):
                for i in os.listdir(self.dir_download):
                    print("Removendo arquivo ", i)
                    os.remove(f'{self.dir_download}/{i}')

        else:
            print("Diretório não existe")
            # Cria diretório que não existia
            os.mkdir(self.dir_download)

        chrome_options= webdriver.ChromeOptions()
        prefs = {'download.default_directory':self.dir_download}
        chrome_options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(options=chrome_options)

        print('Passo 1 de 4 - Extração dos arquivos zipados da CVM')
        driver.get(self.url)

        ano_ini = int(self.ini[-4:])
        ano_fim = int(self.fim[-4:])
        c = 2 + (ano_ini - 2011)
        cf = 2 + (ano_fim - 2011)

        while True:
            try:
                driver.find_element(By.XPATH, f'/html/body/div[1]/pre/a[{c}]').click()
                time.sleep(3)
                c += 1
                if c == cf:
                    break
            except:
                break

        while True:
            if len([x for x in os.listdir(self.dir_download) if ('.zip.crdownload' in x)]) > 0:
                pass
            else:
                break

        print('Passo 2 de 4 - Download dos arquivos concluído')
        print('Passo 3 de 4 - Iniciando processo de Unzip das pastas')

        extension = '.zip'
        for item in os.listdir(self.dir_download): # Loop para percorrer todos os arquivos que estão na pasta "files"
            if item.endswith(extension): # Verifica se o arquivo é .zip
                #self.dir_download = os.path.abspath(self.dir_download)
                zipref = zipfile.ZipFile(self.dir_download + '\\' + item) # Criando objeto zipfile
                zipref.extractall(self.dir_download) # Extraindo arquivo para diretório
                zipref.close() # Fecha o arquivo zipado
                os.remove(self.dir_download + '\\' + item) # Deleta cada um dos zips

        print('Passo 4 de 4 - Todos os arquivos .csv foram unzipados')
        driver.quit()


    def import_df(self, dfp, extract = True):
        if extract == True:
            self.scraping_df()
        elif extract == False:
            pass
        
        print('Iniciando processo de unificação dos balanços conforme período selecionado')
        filename = []

        ini = self.ini
        fim = self.fim

        ano_ini = ini[-4:]
        ano_fim = fim[-4:]

        if ano_ini == ano_fim:
            periodo = [ano_ini, None]
        else:
            periodo = [ano_ini, ano_fim]
            lista_anos = list(range(int(periodo[0]), int(periodo[1])+1))


        for d in dfp:
            for p in lista_anos:
                filename += [x for x in os.listdir(self.dir_download) if (d in x) and (str(p) in x)]

        for i in filename:
            df = pd.read_csv(f"{self.dir_download}\\{i}", sep = ';', encoding = 'cp1252')
            try:
                df_concat = pd.concat([df_concat,df], ignore_index = True)
            except:
                df_concat = df

        df_concat['DT_INI_EXERC'] = pd.to_datetime(df_concat['DT_INI_EXERC'], format = ('%Y-%m-%d'))
        df_concat['DT_FIM_EXERC'] = pd.to_datetime(df_concat['DT_FIM_EXERC'], format = ('%Y-%m-%d'))

        ini_log =  f'{ini[-4:]}-{ini[3:5]}-{ini[0:2]}'
        fim_log =  f'{fim[-4:]}-{fim[3:5]}-{fim[0:2]}'

        df_final = df_concat.loc[(df_concat['DT_FIM_EXERC'] >= ini_log) & (df_concat['DT_FIM_EXERC'] <= fim_log)]

        print('Dataframe finalizado')
        return df_final

In [8]:
cvm = CVM_Extractor('01/01/2021','01/01/2024')

In [9]:
df_dre = cvm.import_df(['DRE'], True)

Diretório Existe
Removendo arquivo  itr_cia_aberta_2021.csv
Removendo arquivo  itr_cia_aberta_2022.csv
Removendo arquivo  itr_cia_aberta_2023.csv
Removendo arquivo  itr_cia_aberta_BPA_con_2021.csv
Removendo arquivo  itr_cia_aberta_BPA_con_2022.csv
Removendo arquivo  itr_cia_aberta_BPA_con_2023.csv
Removendo arquivo  itr_cia_aberta_BPA_ind_2021.csv
Removendo arquivo  itr_cia_aberta_BPA_ind_2022.csv
Removendo arquivo  itr_cia_aberta_BPA_ind_2023.csv
Removendo arquivo  itr_cia_aberta_BPP_con_2021.csv
Removendo arquivo  itr_cia_aberta_BPP_con_2022.csv
Removendo arquivo  itr_cia_aberta_BPP_con_2023.csv
Removendo arquivo  itr_cia_aberta_BPP_ind_2021.csv
Removendo arquivo  itr_cia_aberta_BPP_ind_2022.csv
Removendo arquivo  itr_cia_aberta_BPP_ind_2023.csv
Removendo arquivo  itr_cia_aberta_DFC_MD_con_2021.csv
Removendo arquivo  itr_cia_aberta_DFC_MD_con_2022.csv
Removendo arquivo  itr_cia_aberta_DFC_MD_con_2023.csv
Removendo arquivo  itr_cia_aberta_DFC_MD_ind_2021.csv
Removendo arquivo  itr_cia

In [5]:
df_dre

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8.208426e+06,S
3,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-2.938607e+06,S
5,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.01,Energia comprada para revenda,-4.993160e+05,N
7,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.02,Encargos sobre uso da rede elétrica,-4.556680e+05,N
9,00.001.180/0001-26,2021-03-31,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.02.03,Combustível para produção de energia elétrica,-5.373370e+05,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189047,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-07-01,2023-09-30,3.99.02,Lucro Diluído por Ação,0.000000e+00,N
1189048,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-01-01,2022-09-30,3.99.02.01,ON,7.384000e-01,N
1189049,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-07-01,2022-09-30,3.99.02.01,ON,2.091000e-01,N
1189050,97.837.181/0001-47,2023-09-30,1,DEXCO S.A.,21091,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-09-30,3.99.02.01,ON,7.435000e-01,N


In [6]:
df_dre.loc[df_dre['CNPJ_CIA'] == '00.000.000/0001-91']

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
152687,00.000.000/0001-91,2021-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01,Receitas de Intermediação Financeira,31186377.0,S
152689,00.000.000/0001-91,2021-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01.01,Operações de Crédito,22608968.0,N
152691,00.000.000/0001-91,2021-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01.02,Operações de Arrendamento Mercantil,0.0,N
152693,00.000.000/0001-91,2021-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01.03,Resultado de Operações com TVM,6707236.0,N
152695,00.000.000/0001-91,2021-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2021-01-01,2021-03-31,3.01.04,Resultado de IFD,1805559.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953725,00.000.000/0001-91,2023-09-30,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-07-01,2023-09-30,3.99.01,Lucro Básico por Ação,0.0,N
953726,00.000.000/0001-91,2023-09-30,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-01-01,2022-09-30,3.99.02,Lucro Diluído por Ação,0.0,N
953727,00.000.000/0001-91,2023-09-30,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2022-07-01,2022-09-30,3.99.02,Lucro Diluído por Ação,0.0,N
953728,00.000.000/0001-91,2023-09-30,1,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-09-30,3.99.02,Lucro Diluído por Ação,0.0,N
